In [4]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import EfficientNetB0
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.optimizers import adam_v2
from keras import backend as keras
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, LearningRateScheduler

import cv2
from cv2 import imread, createCLAHE # read and equalize images

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from skimage.util import montage as montage2d

import os
from glob import glob
from tqdm import tqdm
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
!mkdir /home/TBX11K/HEF

!mkdir /home/TBX11K/HEF/train
!mkdir /home/TBX11K/HEF/val
!mkdir /home/TBX11K/HEF/test

!mkdir /home/TBX11K/HEF/train/sick
!mkdir /home/TBX11K/HEF/train/health
!mkdir /home/TBX11K/HEF/train/tb

!mkdir /home/TBX11K/HEF/val/sick
!mkdir /home/TBX11K/HEF/val/health
!mkdir /home/TBX11K/HEF/val/tb


!mkdir /home/TBX11K/HEF/test/Montgomery
!mkdir /home/TBX11K/HEF/test/Montgomery/health
!mkdir /home/TBX11K/HEF/test/Montgomery/tb
!mkdir /home/TBX11K/HEF/test/Montgomery/sick

!mkdir /home/TBX11K/HEF/test/Shenzhen
!mkdir /home/TBX11K/HEF/test/Shenzhen/health
!mkdir /home/TBX11K/HEF/test/Shenzhen/tb
!mkdir /home/TBX11K/HEF/test/Shenzhen/sick

In [2]:
!mkdir /home/TBX11K/HEF/test/DA_DB
!mkdir /home/TBX11K/HEF/test/DA_DB/health
!mkdir /home/TBX11K/HEF/test/DA_DB/tb
!mkdir /home/TBX11K/HEF/test/DA_DB/sick

In [6]:
from multiprocessing import Pool
from collections import OrderedDict, defaultdict

def histogram(data):
    '''Generates the histogram for the given data.
    Parameters:
        data: data to make the histogram.
    Returns: histogram, bins.
    '''

    pixels, count = np.unique(data, return_counts=True)
    hist = OrderedDict()

    for i in range(len(pixels)):
        hist[pixels[i]] = count[i]

    return np.array(list(hist.values())), np.array(list(hist.keys()))

def convertToHEF(img):
    '''Runs the algorithm for the image.'''

    # HF part
    img_fft = np.fft.fft2(img)  # img after fourier transformation
    img_sfft = np.fft.fftshift(img_fft)  # img after shifting component to the center

    m, n = img_sfft.shape
    filter_array = np.zeros((m, n))

    for i in range(m):
        for j in range(n):
            filter_array[i, j] = 1.0 - np.exp(- ((i-m / 2.0) ** 2 + (j-n / 2.0) ** 2) / (2 * (150 ** 2))) #change limit 80

    high_filter = 0.5 + 0.75*filter_array

    img_filtered = high_filter * img_sfft
    img_hef = np.real(np.fft.ifft2(np.fft.fftshift(img_filtered)))  # HFE filtering done

    # HE part
    # Building the histogram
    hist, bins = histogram(img_hef)
    # Calculating probability for each pixel
    pixel_probability = hist / hist.sum()
    # Calculating the CDF (Cumulative Distribution Function)
    cdf = np.cumsum(pixel_probability)
    cdf_normalized = cdf * 255
    hist_eq = {}
    for i in range(len(cdf)):
        hist_eq[bins[i]] = int(cdf_normalized[i])

    for i in range(m):
        for j in range(n):
            img[i][j] = hist_eq[img_hef[i][j]]

    return img.astype(np.uint8)



def process_clahe(res):
    filename = res['filename']
    dest_file = res['dest_file']
    image = cv2.imread(filename,cv2.IMREAD_GRAYSCALE)
    hef_image = convertToHEF(image)
    cv2.imwrite(dest_file,hef_image)
    return True

def process_image_files(img_files,dest_dir):
    res = []
    for filename in img_files:
        dest_file = os.path.join(dest_dir,os.path.basename(filename))
        row = {'filename':filename,'dest_file':dest_file}
        res.append(row)
    
    workers = os.cpu_count()
    with Pool(workers) as p:
        p.map(process_clahe, res)  

In [4]:
TBX_DIR = '/home/TBX11K'
TBX_DEST_DIR = '/home/TBX11K/HEF'
for split_dir in ['train','val']:
    for class_dir in ['health','sick','tb']:
        file_path = os.path.join(TBX_DIR,split_dir,class_dir,'*.png')
        img_files = glob(file_path)
        dest_dir = os.path.join(TBX_DEST_DIR,split_dir,class_dir)
        #dest_file = os.path.join(dest_dir,os.path.basename(img_file))
        #process_clahe({'filename':img_file,'dest_file':dest_file})
        process_image_files(img_files,dest_dir)
        

In [5]:
TBX_DIR_TEST = '/home/TBX11K/test'
TBX_DEST_DIR_TEST = '/home/TBX11K/HEF/test'
for split_dir in ['Montgomery','Shenzhen']:
    for class_dir in ['health','sick','tb']:
        file_path = os.path.join(TBX_DIR_TEST,split_dir,class_dir,'*.png')
        img_files = glob(file_path)
        dest_dir = os.path.join(TBX_DEST_DIR_TEST,split_dir,class_dir)
        process_image_files(img_files,dest_dir)

In [7]:
TBX_DIR_TEST = '/home/TBX11K/test'
TBX_DEST_DIR_TEST = '/home/TBX11K/HEF/test'
for split_dir in ['DA_DB']:
    for class_dir in ['tb']:
        file_path = os.path.join(TBX_DIR_TEST,split_dir,class_dir,'*.png')
        img_files = glob(file_path)
        dest_dir = os.path.join(TBX_DEST_DIR_TEST,split_dir,class_dir)
        
        #dest_file = os.path.join(dest_dir,os.path.basename(img_files[0]))
        #print(img_files[0],dest_file)
        #break
        #process_clahe({'filename':img_file,'dest_file':dest_file})
        process_image_files(img_files,dest_dir)